In [1]:
'''
Scrapy shell Xpath testing command:


scrapy shell -s USER_AGENT="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36" "https://www.chartable.com/charts/spotify/united-states-of-america-top-podcasts"  

"https://chartable.com/podcasts/the-joe-rogan-experience" 
'''
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import csv
from datetime import date
import getpass

In [2]:
#Add User agent to scrape the site
from selenium.webdriver.chrome.options import Options
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36" 
)

In [6]:
from configparser import ConfigParser
cfg = ConfigParser(interpolation=None) 
cfg.read('conf.ini')
login_conf = cfg['chartable']
user_email = login_conf['email'] 
passwd = login_conf['passwd']

In [5]:
# Test Login to Chartable


driver = webdriver.Chrome(options=opts)
driver.get('https://www.chartable.com/sign_in')

email = driver.find_element_by_id('Email')

password = driver.find_element_by_id('Password')

email.send_keys(user_email)
password.send_keys(passwd)

driver.find_element_by_name('commit').click()
time.sleep(5)
driver.close()


In [17]:
driver = webdriver.Chrome(options=opts)
# driver.get('https://www.chartable.com/sign_in')

# email = driver.find_element_by_id('Email')

# password = driver.find_element_by_id('Password')

# email.send_keys(user_email)
# password.send_keys(passwd)

# driver.find_element_by_name('commit').click()
# time.sleep(5)


# Start .csv file
csv_file = open('spotify_podcasts.csv', 'w', encoding='utf-8', newline='')
writer = csv.writer(csv_file)

#Open Google Chrome bot
start_page = 'https://www.chartable.com/charts/spotify/united-states-of-america-top-podcasts'
driver.get(start_page)

#Get ranks of all on the page
ranks_elems = driver.find_elements_by_xpath('//div[@class = "b header-font f2 tc"]')
ranks = [int(rank.text) for rank in ranks_elems]
rows = driver.find_elements_by_xpath('//td[@class = "pv2 ph1"]')
for row in rows:
    podcast_dict = {}
    try:
        url = row.find_element_by_xpath('./div[@class = "title f3"]/a').get_attribute('href')
    except:
        ranks.pop(0)
        continue
    if row.text.find('\n') ==-1:
        name = row.text
        network = 'Unaffiliated'
    else:
        network, name= row.text.split('\n')
    
    podcast_dict['rank']=ranks.pop(0)
    podcast_dict['name'] = name
    podcast_dict['network'] = network
    
#Transfer to the podcast page
    time.sleep(5)
    driver.get(url)
    
    try:
        find_genre = driver.find_elements_by_xpath('//div[@class = "links bg-white pa3 br2 b--near-white ba f6"]//div/a[contains(@href,"genre")]')
        try:
            genre = find_genre.text
        except:
            genre = [i.text for i in find_genre]
    except:
        genre = 'Unknown'
    
    stars_ratings = driver.find_element_by_xpath('//div[@class = "gray"]').text
    stars, ratings = stars_ratings.split(' stars from ')
    try:
        ratings = int(ratings.replace(',','').replace('ratings',''))
    except:
        ratings = int(ratings.replace('ratings',''))
    
    
    
    podcast_dict['genre'] = genre
    podcast_dict['stars'] = stars
    podcast_dict['ratings'] = ratings
    
    
    
    episodes_url = driver.find_element_by_xpath('//div[@class = "link mb2"]/a').get_attribute('href')
    time.sleep(5)
    driver.get(episodes_url)
    
    episodes = driver.find_elements_by_xpath('//div[@class = "mb4"]')
    for episode in episodes:
        episode_date = episode.find_element_by_xpath('.//div[@class = "mt2 f6 light-silver"]').text
        episode_date = episode_date.replace('Published ','')
        podcast_dict['episode_date'] = episode_date

        writer.writerow(podcast_dict.values())   
    

    
csv_file.close()
driver.close()

## Run line below if error leaves csv/driver open

In [135]:
try:
    csv_file.close()
    driver.close()
except:
    driver.close()

In [ ]:
df_genre = pd.merge(df_genre.drop('genres', axis=1), genre, how='right', left_index=True, right_on='index')
df_genre.head()

### Working on getting names

In [ ]:
wait_ranks =WebDriverWait(driver, 10)
podcast_dict = {}
ranks = wait_ranks.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class = "b header-font f2 tc"]')))
for rank in ranks:
    num_rank = review.find_element_by_xpath('//div[@class = "b header-font f2 tc"]').text
    podcast_dict['rank'] = num_rank

## Data reading/cleaning process

In [26]:
spotify = pd.read_csv('spotify_podcasts.csv',names=['rank','name','network','genre','stars','ratings','episode_date'])
spotify.head(10)
spotify['episode_date'] = pd.to_datetime(spotify['episode_date'])
spotify['episode_day_of_week'] = spotify['episode_date'].dt.day_name()
spotify.genre = spotify['genre'].apply(lambda s: s[2:-2].replace("'",'').split(', '))

def list_unpacking(df,colname):
    index = 0
    list_ = []
    for item in df[colname]:
        list_.extend(map(lambda x: [index,x],item))
        index +=1
    new_df = pd.DataFrame(list_, columns = ['index', colname + 's'])
    df = df.merge(new_df,left_index = True, right_on = 'index')
    df.drop(['genre','index'],axis = 1, inplace = True)
    return df
                                            

genre_unpacked = list_unpacking(spotify,'genre')
import numpy as np
genre_unpacked['my_index'] = genre_unpacked.stars**2/np.log(genre_unpacked.ratings)

spotify.drop
genre_unpacked.head(10)
#then join from index


,rank,name,network,stars,ratings,episode_date,episode_day_of_week,genres,my_index
0,1,The Joe Rogan Experience,Joe Rogan,4.7,261177,2020-11-27,Friday,Comedy,1.771032
1,1,The Joe Rogan Experience,Joe Rogan,4.7,261177,2020-11-27,Friday,Society & Culture,1.771032
2,1,The Joe Rogan Experience,Joe Rogan,4.7,261177,2020-11-25,Wednesday,Comedy,1.771032
3,1,The Joe Rogan Experience,Joe Rogan,4.7,261177,2020-11-25,Wednesday,Society & Culture,1.771032
4,1,The Joe Rogan Experience,Joe Rogan,4.7,261177,2020-11-24,Tuesday,Comedy,1.771032
5,1,The Joe Rogan Experience,Joe Rogan,4.7,261177,2020-11-24,Tuesday,Society & Culture,1.771032
6,1,The Joe Rogan Experience,Joe Rogan,4.7,261177,2020-11-20,Friday,Comedy,1.771032
7,1,The Joe Rogan Experience,Joe Rogan,4.7,261177,2020-11-20,Friday,Society & Culture,1.771032
8,1,The Joe Rogan Experience,Joe Rogan,4.7,261177,2020-11-19,Thursday,Comedy,1.771032
9,1,The Joe Rogan Experience,Joe Rogan,4.7,261177,2020-11-19,Thursday,Society & Culture,1.771032


In [27]:
import pandas as pd
spotify_top50 = pd.read_csv('spotify_top50podcasts.csv',names = ['rank','name','network'])
spotify_top50.head()

,rank,name,network
0,1,The Joe Rogan Experience,Joe Rogan
1,2,The Daily,The New York Times
2,3,Crime Junkie,audiochuck
3,4,NPR News Now,NPR
4,5,Up First,NPR
